In [1]:
# Imports
import numpy as np
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest
from hftbacktest import Recorder
from hftbacktest.stats import LinearAssetRecord
import pandas as pd

# Data Input paths
preprocessed_data_path = "../data/daily_processed"
daily_eod_snapshots = "../data/snapshots" # EOD = End Of Day 

In [2]:
day_start = 2
day_end = 3
data = []

for i in range(day_start, day_end):
    day = str(i) if i > 9 else "0" + str(i)
    day_file = f"../data/daily_processed/deribit_eth_perp_2025-01-{day}.npz"
    
    day_latency = f"../data/latencies/latency_2025-01-{day}_latency.npz"
    data.append(day_file)

day_start_str = str(day_start - 1) if day_start > 10 else "0" + str(day_start - 1)
eod = np.load(f"../data/snapshots/deribit_eth_perp_2025-01-{day_start_str}_eod.npz")['data']

stats = pd.read_csv("../data-generated/depth_stats/daily_stats_25_merged.csv").to_numpy()


In [3]:
from src.others.backtest_generate_features import build_rf_features


# Taken from https://support.deribit.com/hc/en-us/articles/25944746248989-Fees
MAKER_FEE = -0.0001
TAKER_FEE =  0.0005    

gamma = 0.01143319447439636
delta = 1
adj1 = 0.9314120055506834
adj2 = 0.43081606500331926

max_position=50
asset = (
BacktestAsset()
    .data(data)
    .initial_snapshot(eod)
    .linear_asset(1.0)
    # .intp_order_latency(latencies, True)
    .constant_latency(10000, 10000) # Constant latency model (nanoseconds) values inspired from https://roq-trading.com/docs/blogs/2023-01-12/deribit/
    .risk_adverse_queue_model()
    # .power_prob_queue_model(2.0)
    .no_partial_fill_exchange()
    .trading_value_fee_model(MAKER_FEE, TAKER_FEE)
    .tick_size(0.01) # Tick size of this asset: minimum price increasement
    .lot_size(1) # Lot size of this asset: minimum trading unit
    # .roi_lb(0.0) # Sets the lower bound price for the range of interest in the market depth.
    # .roi_ub(3000.0) # Sets the upper bound price for the range of interest in the market depth.
    .last_trades_capacity(10000)
)

hbt = HashMapMarketDepthBacktest([asset])

n_trading_days = day_end - day_start
recorder = Recorder(1, n_trading_days* 1_000_000)

features, mid_prices, t = build_rf_features(hbt, 2, stats)
hbt.close()

np.save("../data/features/normalized_01_jan.npy", features)
np.save("../data/features/mid_prices_01_jan.npy", mid_prices)

got here


In [4]:
stats

array([[3.34011515e+03, 1.07219772e+01, 3.00603849e+04, ...,
        1.06755936e+01, 1.91794201e+04, 2.46277385e+04],
       [3.44102787e+03, 3.44609639e+01, 3.24759358e+04, ...,
        3.45532493e+01, 2.03756267e+04, 2.80579426e+04],
       [3.51886512e+03, 6.63322083e+01, 4.42608279e+04, ...,
        6.62780423e+01, 2.31986311e+04, 2.57416562e+04],
       ...,
       [3.11849339e+03, 2.18191748e+01, 3.10509825e+04, ...,
        2.18603157e+01, 2.45695303e+04, 5.02425575e+04],
       [3.22316908e+03, 4.70695717e+01, 3.30069553e+04, ...,
        4.70435578e+01, 2.67825707e+04, 5.41798893e+04],
       [3.31608227e+03, 5.39999857e+01, 2.62828821e+04, ...,
        5.39238616e+01, 2.56036233e+04, 4.11591841e+04]])